# SOLAR POWER USECASE

This tutorial leverage tha analytical capabilities of snowflake to understand how efficient or not efficient roofs of buildings are based on how much of the roof is 'pitched' and at what angle the pitch is at.  We will be looking at a sub section of the buildings data which we have just added search optimisation to.  We will allow the user to search within a distance range of a postcode.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
import pydeck as pdk
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


# 1. Initial Data Transformation

The dataset we will be working on is using a sample set of buildings provided by Ordnance Survey.  The sample set spans over 3 areas within the United Kingdom.

We will do an initial transformation by geocoding each building to the Ordanence Survey urban extents - this will allow the user to filter each area by Urban area.  We will also fetch the UPRN (Unique Property Reference number) and assign it to each building with a spatial join. (**ST_INTERSECTS**)

In [ ]:
USE WAREHOUSE XX_LARGE_HEAVY_LIFT;
CREATE OR REPLACE TABLE DEFAULT_SCHEMA.BUILDINGS_WITH_ROOF_SPECS AS 

SELECT A.*,B.UPRN FROM (
select A.*,B.NAME1_TEXT from ORGDATACLOUD$INTERNAL$OS_BUILDING_SAMPLE_DATA.ORDNANCE_SURVEY_SAMPLE_DATA.BUILDINGS_WITH_ROOF_SPECS A

INNER JOIN

URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_EXTENTS_TBL B

ON

ST_WITHIN(A.GEOGRAPHY,B.GEOGRAPHY) 

ORDER BY NAME1_TEXT

)A INNER JOIN 

UNIQUE_PROPERTY_REFERENCE_NUMBERS__GREAT_BRITAIN_OPEN_UPRN.PRS_OPEN_UPRN_SCH.PRS_OPEN_UPRN_TBL B ON 

ST_INTERSECTS(A.GEOGRAPHY,B.GEOGRAPHY)




;

ALTER WAREHOUSE XX_LARGE_HEAVY_LIFT SUSPEND;

USE WAREHOUSE DEFAULT_WH;

# 2. Search Optimisation
For quick querying of geographic fields, alter the table to add serch optimisation to the geography field

Finally, now we have created the table we will add search optimisation onto the Geography field

In [ ]:
ALTER TABLE DEFAULT_SCHEMA.BUILDINGS_WITH_ROOF_SPECS ADD SEARCH OPTIMIZATION ON GEO(GEOGRAPHY);

# 3.  Spatial Filter
## Filtering a Postcode selectbox list 

After town, the next filtering will be based on distance of a postcode.  The places dataset provided by Ordnance survey provides a complete list of postcodes with corresponding centroids.  There are over a million postcodes - most of which is irellevant for this sample dataset.  

Therefore we need to filter the postcodes which are within the extents where the sample buildings are located. 

You will note that **ST_WITHIN** is used to ensure only the postcodes are included that fit the relevant urban areas.  The postcode list will also include the geography details which we will then use to calculate distance

In addition, we have created a postcode sector column as this will be used to join to weather data.

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE DEFAULT_SCHEMA.TBL1 AS

SELECT C.GEOGRAPHY,C.NAME1_TEXT FROM (

URBAN_EXTENTS_FOR_CITIES_TOWNS_AND_VILLAGES__GREAT_BRITAIN_OPEN_BUILT_UP_AREAS.PRS_OPEN_BUILT_UP_AREAS_SCH.PRS_OPEN_BUILT_UP_EXTENTS_TBL C

    INNER JOIN 

    (SELECT DISTINCT NAME1_TEXT NAME1_TEXT FROM DEFAULT_SCHEMA.BUILDINGS_WITH_ROOF_SPECS)D 
    
    ON C.NAME1_TEXT =D.NAME1_TEXT


) ;

CREATE OR REPLACE TEMPORARY TABLE DEFAULT_SCHEMA.TBL2 AS

Select NAME1, GEOGRAPHY FROM
POSTCODES_PLACE_NAMES_AND_ROAD_NUMBERS__GREAT_BRITAIN_OPEN_NAMES.PRS_OPEN_NAMES_SCH.PRS_OPEN_NAMES_TBL WHERE LOCAL_TYPE = 'Postcode';




CREATE OR REPLACE TABLE DEFAULT_SCHEMA.POSTCODES AS 

SELECT A.NAME1,A.GEOGRAPHY, B.NAME1_TEXT, CONCAT(SPLIT(A.NAME1,' ')[0],'_',LEFT(SPLIT(A.NAME1,' ')[1],1)) PC_SECT

FROM  DEFAULT_SCHEMA.TBL2 A


    INNER JOIN 

    DEFAULT_SCHEMA.TBL1 B

    ON ST_WITHIN(A.GEOGRAPHY,B.GEOGRAPHY);


USE WAREHOUSE DEFAULT_WH;

SELECT COUNT(*) FROM DEFAULT_SCHEMA.POSTCODES;

Below you can seee the postcodes applied to the filter.  The user chooses the town first, then only postcodes for that town AND only where there are buildings will appear.  The distance is used for a further filter, which will only retrieve buildings with an Xm distance.  **ST_D_WITHIN** is used for this calculation 

In [ ]:
BUILDINGS = session.table('DEFAULT_SCHEMA.BUILDINGS_WITH_ROOF_SPECS')
col1,col2,col3 = st.columns(3)

with col1:
    filter = BUILDINGS.select('NAME1_TEXT').distinct()
    filter = st.selectbox('Choose Town:',filter)
with col2:
    postcodes = session.table('default_schema.POSTCODES').filter(col('NAME1_TEXT')==filter)
    postcodef = st.selectbox('Postcode:',postcodes)
with col3:
    distance = st.number_input('Distance in M:', 20,2000,500)

st.divider()

selected_point = session.table('DEFAULT_SCHEMA.POSTCODES').filter(col('NAME1')==postcodef).select('GEOGRAPHY','PC_SECT')

In [ ]:
BUILDINGS = BUILDINGS.filter(col('NAME1_TEXT')==filter)

BUILDINGS = BUILDINGS.join(selected_point.with_column_renamed('GEOGRAPHY','SPOINT'),
                           call_function('ST_DWITHIN',selected_point['GEOGRAPHY'],
                                        BUILDINGS['GEOGRAPHY'],distance)).drop('SPOINT')

st.write(BUILDINGS.limit(5))

# 4. Time Series Data
### Dataset 2 Solar Elevation angle
The Met Office calculate the solar elevation angle which is the angle referring to where the sun is in the sky.  This is used to capture how much energy can be consumed from potential solar panels.  You will note this data is by hour and postcode sector

In [ ]:
CREATE OR REPLACE TABLE DEFAULT_SCHEMA.SOLAR_ELEVATION AS SELECT * EXCLUDE "Solar_elevation_angle" ,round("Solar_elevation_angle"::DECIMAL(7,3),3) "Solar_elevation_angle" FROM ORGDATACLOUD$INTERNAL$OS_BUILDING_SAMPLE_DATA.ORDNANCE_SURVEY_SAMPLE_DATA.SOLAR_ELEVATION;

select * from DEFAULT_SCHEMA.solar_elevation limit 1000

We will call the resulting table in a dataframe and use a data filter to allow the user to pick a specific date in the year.

In [ ]:
SOLAR_ELEVATION_DF = session.table('DEFAULT_SCHEMA.SOLAR_ELEVATION')
selected_date = st.date_input('Date:',datetime.date(2024,1, 1),datetime.date(2024,1,1),datetime.date(2024,12,31))

SOLAR_ELEVATION_DF_FILTERED = SOLAR_ELEVATION_DF.filter(call_function('date',col('"Validity_date_and_time"'))==selected_date)

# 5. Variable Factors 
### Loss of Power due to where the roofs are pointing
Here, we are making assumptions of how much power is lossed due to the slopey roof being aimed away from the sun.  

A south facing slopey roof can generate the most energy, whilst a north facing roof generates the least energy.  Sliders are used to make the each factor dynamic.

In [ ]:
with st.container():
    south_facing = st.slider('South Facing:',0.0,1.0,1.0)
    south_east_facing = st.slider('South East Facing:',0.0,1.0,0.90)
    south_west_facing = st.slider('South West Facing:',0.0,1.0,0.80)
    west_facing = st.slider('West Facing:',0.0,1.0,0.70)
    north_east_facing = st.slider('North East Facing:',0.0,1.0,0.60)
    north_west_facing = st.slider('North West Facing:',0.0,1.0,0.60)
    east_facing = st.slider('East Facing:',0.0,1.0,0.30)
    north_facing = st.slider('North Facing:',0.0,1.0,0.10)

    solar_panel_angle = st.slider('Solar Panel Elevation Angle:',0,90,30)

    solar_joules_per_s = 1000
    panel_rating_W = 300
    size = 1.89


Next,we apply these factors to each of the areas that have directional slopes.

In [ ]:
BUILDINGS = BUILDINGS.with_column('DIRECT_IRRADIANCE_M2',
                                  col('ROOFSHAPEASPECT_AREAFLAT_M2')+
                                  col('ROOFSHAPEASPECT_AREAFACINGNORTH_M2')*north_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGSOUTH_M2')*south_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGEAST_M2')*east_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGWEST_M2')*west_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGNORTHWEST_M2')*north_east_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGNORTHEAST_M2')*north_west_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGSOUTHEAST_M2')*south_east_facing +
                                 col('ROOFSHAPEASPECT_AREAFACINGSOUTHWEST_M2')*south_west_facing)

Below is a dataframe summary, where you have:
-   **Direct Irradiance** - the area exposed by direct sun light with sloping factors taking into account

- **Total Area** - The total area of all roof tops

In [ ]:
##### GROUP BUILDINGS TO
SOLAR_BUILDINGS_SUM = BUILDINGS.agg(sum('DIRECT_IRRADIANCE_M2').alias('DIRECT_IRRADIANCE_M2'),
                                   sum('GEOMETRY_AREA_M2').alias('TOTAL_AREA'))

st.dataframe(SOLAR_BUILDINGS_SUM)

# 6. Join Time and Spatial dataset together
We will now join the angle of the sun from the weather data with the roof tops dataframe. We will display the results as a time based line chart.  You will notice that there no direct irradiance when it's dark.

In [ ]:
SOLAR_BUILDINGS_SUM = BUILDINGS.agg(sum('DIRECT_IRRADIANCE_M2').alias('DIRECT_IRRADIANCE_M2'),
                                   sum('GEOMETRY_AREA_M2').alias('TOTAL_AREA'))

SOLAR_BUILDINGS_SUM = SOLAR_BUILDINGS_SUM.join(SOLAR_ELEVATION_DF_FILTERED.group_by('"Validity_date_and_time"').agg(avg('"Solar_elevation_angle"').alias('"Solar_elevation_angle"')))
SOLAR_BUILDINGS_SUM = SOLAR_BUILDINGS_SUM.with_column('total_energy',when(col('"Solar_elevation_angle"')<0,0).otherwise(col('DIRECT_IRRADIANCE_M2')*cos(radians(lit(solar_panel_angle))-col('"Solar_elevation_angle"'))))

st.line_chart(SOLAR_BUILDINGS_SUM.to_pandas(),y='TOTAL_ENERGY',x='Validity_date_and_time', color='#29B5E8')

# 7. Prepare for Streamlit
Below, the dataframe has been engineered to support the visualisation  - such as specifying columns for tool tip purposes as ell as adding conditional colors for each building

In [ ]:
BUILDINGS_V = BUILDINGS.limit(2000).select('GEOGRAPHY',
                                           'UPRN',
                                           'THEME',
                                           'DESCRIPTION',
                                           col('GEOMETRY_AREA_M2').astype(StringType()).alias('GE'),
                                           'ROOFMATERIAL_PRIMARYMATERIAL',
                                           'ROOFMATERIAL_SOLARPANELPRESENCE',
                                           'ROOFMATERIAL_GREENROOFPRESENCE',
                                           'ROOFSHAPEASPECT_SHAPE',
                                            col('ROOFSHAPEASPECT_AREAPITCHED_M2').astype(StringType()).alias('A'),
                                            col('ROOFSHAPEASPECT_AREAFLAT_M2').astype(StringType()).alias('RF'),
                                            col('DIRECT_IRRADIANCE_M2').astype(StringType()).alias('D'),
                                            div0(col('D'),col('GEOMETRY_AREA_M2')).alias('EFFICIENCY_RATIO'),
                                            when(col('EFFICIENCY_RATIO')>=0.9,[41,181,232]).when(col('EFFICIENCY_RATIO')>=0.8,[17,86,127]).otherwise([255,159,54]).alias('COLOR'),
                                            col('COLOR')[0].alias('R'),
                                            col('COLOR')[1].alias('G'),
                                            col('COLOR')[2].alias('B'))

The tooltip is formatted with all suitable columns added - which includes styling

In [ ]:
tooltip = {
   "html": """ 
   <br> <b>Theme:</b> {THEME} 
   <br> <b>UPRN:</b> {UPRN}
   <br> <b>Description:</b> {DESCRIPTION}
   <br> <b>Roof Material:</b> {ROOFMATERIAL_PRIMARYMATERIAL}
   <br> <b>Solar Panel Presence:</b> {ROOFMATERIAL_SOLARPANELPRESENCE}
   <br> <b>Green Proof Presence:</b> {ROOFMATERIAL_GREENROOFPRESENCE}
   <br> <b>Roof Shape:</b> {ROOFSHAPEASPECT_SHAPE}
   <br> <b>Geometry Area M2:</b> {GE}
   <br> <b>Area Pitched M2:</b> {A}
   <br> <b>Area Flat M2:</b> {RF}
   <br> <b>Direct Irradiance M2:</b> {D}
   <br> <b>Efficiency Ratio:</b> {EFFICIENCY_RATIO}
   """,
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}


The center point is calculated based on extracting the LAT and LON from the selected postcode

In [ ]:
centre = selected_point
centre = centre.with_column('LON',call_function('ST_X',col('GEOGRAPHY')))
centre = centre.with_column('LAT',call_function('ST_Y',col('GEOGRAPHY')))

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]

Finally we will build the pydeck map.  The results will depend on the previous filters applied.

In [ ]:

# Populate dataframe from query
datapd = BUILDINGS_V.to_pandas()

datapd["coordinates"] = datapd["GEOGRAPHY"].apply(lambda row: json.loads(row)["coordinates"])

st.write('Buildings in a town')

# Create data layer - this where the geometry is likely failing - column is now called geometry to match geopandas default
data_layer = pdk.Layer(
    "PolygonLayer",
    datapd,
    opacity=0.8,
    get_polygon="coordinates", 
    filled=True,
    get_fill_color=["R-1","G-1","B-1"],
    get_line_color=[0, 0, 0],
    get_line_width=0.1,
    auto_highlight=True,
    pickable=True,
)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=15,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[data_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)
    
st.pydeck_chart(r, use_container_width=True)




# Streamlit next

Please view the provided **SOLAR_ENERGY_INSIGHTS** streamlit example as a starting point for designing energy consumption analysis for potential solar powered roofs.

Within projects click on Streamlit
Select the **SOLAR_ENERGY_INSIGHTS** project
Run the Streamlit.

You will probably want to refine the calculations or add  more amendments.  There is an option to **Duplicate** the streamlit which will duplicate the app which will be in **Edit** format